## reference
https://www.kaggle.com/robikscube/helper-code-helmet-mapping-deepsort

## import

### private library

In [ ]:
from external_lib.NFLlib.score import NFLAssignmentScorer, check_submission
from external_lib.NFLlib.features import add_track_features

from lib.math.coodinate import generate_rotation_maxrix
from lib.math.coodinate import convert_3d_to_2d

from lib.noglobal import noglobal

### public library

In [ ]:
import numpy as np
import pandas as pd
import itertools
import glob
import os
import cv2
from sklearn.metrics import accuracy_score
from tqdm.auto import tqdm
from multiprocessing import Pool
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
import random

## paramters

In [ ]:
n_debug_samples = 120
random_state = 42
CONF_THRE = 0.3
max_iter = 1000
DIG_STEP = 3
DIG_MAX = 42
base_dir = "/work/data/input/nfl-health-and-safety-helmet-assignment"

debug = True


paramater = {};
paramater["n_debug_samples"] = n_debug_samples
paramater["random_state"] = random_state
paramater["CONF_THRE"] = CONF_THRE
paramater["max_iter"] = max_iter
paramater["DIG_STEP"] = DIG_STEP
paramater["DIG_MAX"] = random_state


## file reading

In [ ]:
labels = pd.read_csv(f"{base_dir}/train_labels.csv")

if debug:
    tracking = pd.read_csv(f'{base_dir}/train_player_tracking.csv')
    helmets = pd.read_csv(f'{base_dir}/train_baseline_helmets.csv')
else:
    tracking = pd.read_csv(f'{base_dir}/test_player_tracking.csv')
    helmets = pd.read_csv(f'{base_dir}/test_baseline_helmets.csv')

## preprocessing

### function definition

In [ ]:
def add_cols(df):
    df['game_play'] = df['video_frame'].str.split('_').str[:2].str.join('_')
    if 'video' not in df.columns:
        df['video'] = df['video_frame'].str.split('_').str[:3].str.join('_') + '.mp4'
    return df

### both train and test

### function difinition

In [ ]:
tracking = add_track_features(tracking)
tracking["z"] = 6.0

if debug:
    helmets = add_cols(helmets)
    labels = add_cols(labels)    
    
    # Select `n_debug_samples` worth of videos to debug with
    sample_videos = labels['video'].drop_duplicates() \
        .sample(n_debug_samples, random_state=random_state).tolist()
    sample_gameplays = ['_'.join(x.split('_')[:2]) for x in sample_videos]
    tracking = tracking[tracking['game_play'].isin(sample_gameplays)]
    helmets = helmets[helmets['video'].isin(sample_videos)]
    labels = labels[labels['video'].isin(sample_videos)]

    
helmets["frame"] = helmets["video_frame"].apply(lambda x:x.split("_")[3])
helmets["video_frame_without_position"] = helmets["video_frame"].apply(lambda x: "_".join( x.split("_")[:2] + [x.split("_")[3]]   )  )
    


for key,hel_each_df in helmets.groupby("game_play"):    
    min_frame = hel_each_df["frame"].astype(int).min()
    max_frame = hel_each_df["frame"].astype(int).max()
    tracking = tracking[tracking["est_frame"] > (min_frame-5)]
    tracking = tracking[tracking["est_frame"] < max_frame+5]  


## main

### first part (basic allocation)

#### function definition

In [ ]:
@noglobal()
def find_nearest(array, value):
    value = int(value)
    array = np.asarray(array).astype(int)
    idx = (np.abs(array - value)).argmin()
    return array[idx]


### [0,1]ではないけど、正規化
@noglobal()
def norm_arr(a):
    a = a-a.min();
    a = a/(a.max());
    return a
    
@noglobal()
def dist(a1,a2):
    return np.linalg.norm(a1-a2);


@noglobal()
def dist_for_different_len(a1,a2,max_iter):
    assert len(a1) >= len(a2), f'{len(a1)}, {len(a2)}'
    
    len_diff = len(a1) - len(a2);

    a2 = norm_arr(a2);
    
    if len_diff == 0:
        a1 = norm_arr(a1);
        return dist(a1,a2),();
    else:
        min_dist = 10000;
        min_delete_idx = None;
        cnt = 0;
        
        del_list = list(itertools.combinations(range(len(a1)),len_diff));    
        
        ## 組み合わせの総数が指定した数より大きい場合、ランダムサンプリングする。。
        if (len(del_list) > max_iter):
            del_list = random.sample(del_list,max_iter);
            
        for delete_idx in del_list:
            this_a1 = np.delete(a1,delete_idx);
            this_a1 = norm_arr(this_a1);
            this_dist = dist(this_a1,a2);
            
            if (min_dist > this_dist):
                min_dist = this_dist
                min_delete_idx = delete_idx;
            
            
        return min_dist,min_delete_idx


@noglobal()
def rotate_arr(u,view, t, isDeg=True):
    if isDeg:
        t = np.deg2rad(t);
        
    if(view == "Endzone"):
        to_Point = np.array([60,0,5])
        from_base_point = np.array([150,0.5*t,6])
        target_axis = "y"
        length_vector = from_base_point - to_Point                            
        rotation_matrix = generate_rotation_maxrix(-1*t,axis=target_axis,isDeg=False);
        
        from_point = to_Point + rotation_matrix@length_vector
        #to_Point = np.array([60,23,5])
        #from_point = np.array([60,23+0.5*t,5])
        
        
    elif(view == "Sideline"):
        to_Point = np.array([60,53.3/2,0])
        from_point = [60 + t,71,40]        
    else:
        raise Exception(f"You can specify only Endzone or Sideline, not {view}");
    
    
    
    return convert_3d_to_2d(u[["x","y","z"]],from_point=from_point,to_point=to_Point)

    

    
@noglobal(excepts = ["rotate_arr"])
def dist_rot(tracking_df, a2,view,DIG_MAX,DIG_STEP,max_iter):
    tracking_df = tracking_df.sort_values('x')
    x = tracking_df['x']
    y = tracking_df['y']
    min_dist = 10000
    min_idx = None
    min_x = None
    
    best_angle = 0;
    
    
    for dig in range(-1*DIG_MAX,DIG_MAX+1,DIG_STEP):                
        
        if (view == "Endzone"):                    
            arr = rotate_arr(tracking_df,view,dig);                
            arr = arr[["vertical","horizon"]].values.T
        elif (view == "Sideline"):
            arr = rotate_arr(tracking_df,view,dig);            
            arr = arr[["horizon","depth"]].values.T
        else:
            raise Exception(f"unknown view mode is specified: {view}")
                                                               
        ##  a2: ヘルメットの座標        
        this_dist, this_idx = dist_for_different_len(np.sort(arr[0]), a2,max_iter)
        if min_dist > this_dist:
            min_dist = this_dist
            min_idx = this_idx
            min_x = arr[0]
            best_angle = dig
        
                            
    tracking_df['x_rot'] = min_x
    player_arr = tracking_df.sort_values('x_rot')['player'].values
    players = np.delete(player_arr,min_idx)
    
    
    return min_dist, players,best_angle  
    

@noglobal()
def mapping_df(args):
    
    ##########################
    ### data difinition ### 
    video_frame, helmets_df = args[:2]
    tracking = args[2]
    parameter = args[3]    
    CONF_THRE = parameter["CONF_THRE"];
    DIG_STEP = parameter["DIG_STEP"];
    DIG_MAX = parameter["DIG_MAX"];
    max_iter = parameter["max_iter"];
            
    gameKey,playID,view,frame = video_frame.split('_')
    
    gameKey = int(gameKey)
    playID = int(playID)
    frame = int(frame)
        
    this_tracking = tracking[(tracking['gameKey']==gameKey) & (tracking['playID']==playID)]
    est_frame = find_nearest(this_tracking.est_frame.values, frame)
    this_tracking = this_tracking[this_tracking['est_frame']==est_frame].reset_index()
    ########################## 
        
    len_this_tracking = len(this_tracking)
    
    helmets_df['center_h_p'] = (helmets_df['left']+helmets_df['width']/2).astype(int)
    helmets_df['center_h_m'] = (helmets_df['left']+helmets_df['width']/2).astype(int)*-1
    helmets_df = helmets_df[helmets_df['conf']>CONF_THRE].copy()        
        
    if len(helmets_df) > len_this_tracking:
        helmets_df = helmets_df.tail(len_this_tracking)
    
    ## x軸でソート
    helmets_df_p = helmets_df.sort_values('center_h_p').copy()
    
    ## 逆順でもソート
    helmets_df_m = helmets_df.sort_values('center_h_m').copy()
    
    
    if view == 'Endzone':        
        this_tracking['x'], this_tracking['y'] = this_tracking['y'].copy(), this_tracking['x'].copy()
        
    helmet_center_a2_p = helmets_df_p['center_h_p'].values
    helmet_center_a2_m = helmets_df_m['center_h_m'].values
    
    
    
    
    min_dist_p, min_detete_idx_p,best_angle_p = dist_rot(this_tracking,helmet_center_a2_p,view,DIG_MAX,DIG_STEP,max_iter)    
    min_dist_m, min_detete_idx_m,best_angle_m = dist_rot(this_tracking,helmet_center_a2_m,view,DIG_MAX,DIG_STEP,max_iter)

    
    if min_dist_p < min_dist_m:
        min_dist = min_dist_p
        min_detete_idx = min_detete_idx_p
        tgt_df = helmets_df_p
        best_angle = best_angle_p
    else:
        min_dist = min_dist_m
        min_detete_idx = min_detete_idx_m
        tgt_df = helmets_df_m
        best_angle = best_angle_m
    
    
    tgt_df['label'] = min_detete_idx
    tgt_df['best_angle'] = best_angle
    return tgt_df[['video_frame','left','width','top','height','label',"best_angle"]]

In [ ]:
submission_df_list = []
df_list = list(helmets.groupby('video_frame'))
df_list = [list(element) + [tracking,paramater] for element in df_list];

### 実行

In [ ]:
from tqdm.notebook import tqdm
from lib.slack import send_massage

from lib.multiprocess import exe_multiprocess_ret_as_list

In [ ]:
counter = 0

submission_output_list = [];

arg_list = [[ key,each_df.reset_index(),tracking,paramater] for key,each_df in helmets.groupby("video_frame")];
submission_output_list = exe_multiprocess_ret_as_list(mapping_df,arg_list)        
submission_df_v2 = pd.concat(submission_output_list)
        
        
debug = True
if debug:
    scorer = NFLAssignmentScorer(labels)
    baseline_score = scorer.score(submission_df_v2)
    print(f"validation score {baseline_score:0.4f}")
    
    

send_massage(token = token,massage = f"finish {baseline_score:0.4f}")

submission_df_v2.to_csv('submission-baseline_upgrade.csv', index=False)


### 

In [ ]:
endline_sub = submission_df_v2[submission_df_v2["video_frame"].str.contains("Endzone")]
endline_labels = labels[labels["video_frame"].str.contains("Endzone")]

#endline_sub
#endline_sub
scorer = NFLAssignmentScorer(endline_labels)
baseline_score = scorer.score(endline_sub)
print(f"validation score {baseline_score:0.4f}")

In [ ]:
endline_sub = submission_df_v2[submission_df_v2["video_frame"].str.contains("Sideline")]
endline_labels = labels[labels["video_frame"].str.contains("Sideline")]

#endline_sub
#endline_sub
scorer = NFLAssignmentScorer(endline_labels)
baseline_score = scorer.score(endline_sub)
print(f"validation score {baseline_score:0.4f}")

In [ ]:
from lib.multiprocess import exe_multiprocess_ret_as_list

submission_df_list = exe_multiprocess_ret_as_list(mapping_df,df_list);

submission_df_v2 = pd.concat(submission_df_list)
#submission_df.to_csv('submission-baseline.csv', index=False)

In [ ]:
debug = True
if debug:
    scorer = NFLAssignmentScorer(labels)
    baseline_score = scorer.score(submission_df_v2)
    print(f"validation score {baseline_score:0.4f}")

## Deep Sort

In [ ]:
from external_lib.deep_sort_pytorch.utils.parser import get_config
from external_lib.deep_sort_pytorch.deep_sort import DeepSort

In [ ]:
def compute_color_for_id(label):
    """
    Simple function that adds fixed color depending on the id
    """
    palette = (2 ** 11 - 1, 2 ** 15 - 1, 2 ** 20 - 1)

    color = [int((p * (label ** 2 - label + 1)) % 255) for p in palette]
    return tuple(color)

def plot_one_box(x, im, color=None, label=None, line_thickness=3):
    # Plots one bounding box on image 'im' using OpenCV
    assert im.data.contiguous, 'Image not contiguous. Apply np.ascontiguousarray(im) to plot_on_box() input image.'
    tl = line_thickness or round(0.002 * (im.shape[0] + im.shape[1]) / 2) + 1  # line/font thickness
    color = color or [random.randint(0, 255) for _ in range(3)]
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(im, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)
    if label: 
        tf = max(tl - 1, 1)  # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(im, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(im, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
    return im

In [ ]:

def deepsort_helmets(video_data,
                     video_dir,
                     deepsort_config='/work/config/deepsort.yaml',
                     plot=False,
                     plot_frames=[]):
    
    # Setup Deepsort
    cfg = get_config()
    cfg.merge_from_file(deepsort_config)    
    deepsort = DeepSort(cfg.DEEPSORT.REID_CKPT,
                        max_dist=cfg.DEEPSORT.MAX_DIST,
                        min_confidence=cfg.DEEPSORT.MIN_CONFIDENCE,
                        nms_max_overlap=cfg.DEEPSORT.NMS_MAX_OVERLAP,
                        max_iou_distance=cfg.DEEPSORT.MAX_IOU_DISTANCE,
                        max_age=cfg.DEEPSORT.MAX_AGE,
                        n_init=cfg.DEEPSORT.N_INIT,
                        nn_budget=cfg.DEEPSORT.NN_BUDGET,
                        use_cuda=True)
    
    # Run through frames.
    video_data = video_data.sort_values('frame').reset_index(drop=True)
    ds = []
    for frame, d in tqdm(video_data.groupby(['frame']), total=video_data['frame'].nunique()):
        d['x'] = (d['left'] + round(d['width'] / 2))
        d['y'] = (d['top'] + round(d['height'] / 2))

        xywhs = d[['x','y','width','height']].values

        cap = cv2.VideoCapture(f'{video_dir}/{myvideo}.mp4')
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame-1) # optional
        
        ##シーケンス
        success, image = cap.read()
        
        # 画像の色の順番を変更（BGR to RGB）
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        confs = np.ones([len(d),])
        clss =  np.zeros([len(d),])
        
        outputs = deepsort.update(xywhs, confs, clss, image)

        
        #if (plot and frame > cfg.DEEPSORT.N_INIT) or (frame in plot_frames):
        #    for j, (output, conf) in enumerate(zip(outputs, confs)): 

        #        bboxes = output[0:4]
        #        id = output[4]
        #        cls = output[5]

        #        c = int(cls)  # integer class
        #        label = f'{id}'
        #        color = compute_color_for_id(id)
        #        im = plot_one_box(bboxes, image, label=label, color=color, line_thickness=2)
        #    fig, ax = plt.subplots(figsize=(15, 10))
        #    video_frame = d['video_frame'].values[0]
        #    ax.set_title(f'Deepsort labels: {video_frame}')
        #    plt.imshow(im)
        #    plt.show()

        preds_df = pd.DataFrame(outputs, columns=['left','top','right','bottom','deepsort_cluster','class'])
        if len(preds_df) > 0:
            # TODO Fix this messy merge
            d = pd.merge_asof(d.sort_values(['left','top']),
                              preds_df[['left','top','deepsort_cluster']] \
                              .sort_values(['left','top']), on='left', suffixes=('','_deepsort'),
                              direction='nearest')
        ds.append(d)
    dout = pd.concat(ds)
    return dout


def add_deepsort_label_col(out):
    # Find the top occuring label for each deepsort_cluster
    sortlabel_map = out.groupby('deepsort_cluster')['label'].value_counts() \
        .sort_values(ascending=False).to_frame() \
        .rename(columns={'label':'label_count'}) \
        .reset_index() \
        .groupby(['deepsort_cluster']) \
        .first()['label'].to_dict()
    # Find the # of times that label appears for the deepsort_cluster.
    sortlabelcount_map = out.groupby('deepsort_cluster')['label'].value_counts() \
        .sort_values(ascending=False).to_frame() \
        .rename(columns={'label':'label_count'}) \
        .reset_index() \
        .groupby(['deepsort_cluster']) \
        .first()['label_count'].to_dict()
    
    out['label_deepsort'] = out['deepsort_cluster'].map(sortlabel_map)
    out['label_count_deepsort'] = out['deepsort_cluster'].map(sortlabelcount_map)

    return out


def score_vs_deepsort(myvideo, out, labels):
    # Score the base predictions compared to the deepsort postprocessed predictions.
    myvideo_mp4 = myvideo + '.mp4'
    labels_video = labels.query('video == @myvideo_mp4')
    scorer = NFLAssignmentScorer(labels_video)
    out_deduped = out.groupby(['video_frame','label']).first().reset_index()
    base_video_score = scorer.score(out_deduped)
    
    out_preds = out.drop('label', axis=1).rename(columns={'label_deepsort':'label'})
    print(out_preds.shape)
    out_preds = out_preds.groupby(['video_frame','label']).first().reset_index()
    print(out_preds.shape)
    deepsort_video_score = scorer.score(out_preds)
    print(f'{base_video_score:0.5f} before --> {deepsort_video_score:0.5f} deepsort')

## main

In [ ]:
submission_df_v2 = pd.read_csv("submission-baseline_upgrade.csv")
submission_df_v2

In [ ]:
# Add video and frame columns to submission.
submission_df = submission_df_v2;

submission_df['video'] = submission_df['video_frame'].str.split('_').str[:3].str.join('_')
submission_df['frame'] = submission_df['video_frame'].str.split('_').str[-1].astype('int')

labels = pd.read_csv(f"{base_dir}/train_labels.csv")

if debug:
    video_dir = f"{base_dir}/train/"
else:
    video_dir = f"{base_dir}/test/"

    
    

# Loop through test videos and apply. If in debug mode show the score change.
outs = []
for myvideo, video_data in tqdm(submission_df.groupby('video'), total=submission_df['video'].nunique()):
    #print(myvideo)
    #print(f'==== {myvideo} ====')
    
    
    if debug:
        # Plot deepsort labels when in debug mode.
        out = deepsort_helmets(video_data, video_dir, plot_frames=[10, 150, 250])
    else:
        out = deepsort_helmets(video_data, video_dir)        
    out = add_deepsort_label_col(out)
    outs.append(out)
    if debug:        
        score_vs_deepsort(myvideo, out, labels)
        
    
submission_deepsort = pd.concat(outs).copy()

In [ ]:
if debug:
    scorer = NFLAssignmentScorer(labels)
    baseline_score = scorer.score(submission_deepsort)
    print(f"validation score {baseline_score:0.4f}")